# Workflows

As noted in the article ["Building effective agents" by Anthropic](https://www.anthropic.com/research/building-effective-agents), workflows can be an effective way to use LLMs to complete complex tasks. Our experience building the first iteration of the "note formatting" feature, small LLMs are not capable of completing such complicated task in one step yet. Our goal in this exploration is to see if breaking down the task in multiple steps of one workflow can significantly improve realability of the system and accuracy.

### Design

1. Original note
2. (Assess note quality)
3. Make concise (input: original note + edited note)
4. Fix code blocks (input: original note + edited note) 
5. Fix math blocks (input: original note + edited note)
6. Fix media (input: original note + edited note)
7. Fix HTML formatting (input: original note + edited note)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from anki_ai.domain.model import Deck, Note
from anki_ai.service_layer.services import get_chat_completion, ChatCompletionsService, strip_tags

from html.parser import HTMLParser
from io import StringIO

In [3]:
deck = Deck("default")
deck.read_txt("../data/Selected Notes v8.txt", exclude_tags=["personal"])
print(f"The deck contains {len(deck)} notes.")

The deck contains 2854 notes.


Let's take a random sample of notes, to iterate more rapidly in our initial exploration. 

In [4]:
deck.shuffle()
sample = deck[:10] # NOTE: this returs a list, should it instead return a Deck?

[Note(guid='df7kE~(DT}', front='Motion to move the cursor [count] WORDS forward', back='"`[count]W`<br><br><img src=""HYaJn.gif"">"', tags=['nvim'], notetype='KaTeX and Markdown Basic (Color)', deck_name='Default'),
 Note(guid='M%}h1HkX0Q', front='"What type of tyres can a hooked rim fit?<br><br><img src=""hooked-rim-fast-fwd-1.jpeg"">"', back='Clincher and (if specified) tubeless-ready tyres', tags=['cycling'], notetype='KaTeX and Markdown Basic (Color)', deck_name='Default'),
 Note(guid='JBmq8%XiDj', front='What command line tool controls the Bluetooth  connections?', back='`bluetoothctl`', tags=['linux'], notetype='KaTeX and Markdown Basic (Color)', deck_name='Default'),
 Note(guid='sOuU_5Cjji', front='What key returns the `^` in the shifted state?', back='"`6`<br><br><img src=""paste-35252f02ada229d38491a63d20fa2ea9e8a5f0f8.jpg"">"', tags=['keyboard'], notetype='KaTeX and Markdown Basic (Color)', deck_name='Default'),
 Note(guid='dbqggvnyg(', front='What does a high negative TSB nu

Now that we have a small dataset we can use, let's start working on creating those steps in the workflow. 

In [5]:
chat = get_chat_completion()

In [6]:
def apply_transform(
    system_msg: str,
    user_msg: str,
    note, 
    edited_note=None,
    chat: ChatCompletionsService = chat,
    json_mode: bool = False,
):
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user"  , "content": user_msg(note, edited_note)},
    ]

    if json_mode:
        extra_body = {
            "guided_json": Note.model_json_schema(),
            "guided_whitespace_pattern": r"[\n\t ]*",
        }
    else:
        extra_body = {}

    chat_response = chat.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        messages=messages,
        temperature=0,
        extra_body=extra_body,
    )

    print("\n#######################\n")
    print(f"Front: {strip_tags(note.front)}\nBack: {strip_tags(note.back)}\nTags: {note.tags}\n")
    if json_mode:
        json_data = chat_response.choices[0].message.content
        new_note = Note.model_validate_json(json_data)
        print(f"Front: {strip_tags(new_note.front)}\nBack: {strip_tags(new_note.back)}\nTags: {new_note.tags}")
    else:
        print(chat_response.choices[0].message.content)

    return (note, new_note)

In [8]:
system_msg = r"""Optimize this Anki note:
- Concise, simple, distinct
- Follow format rules
- Markdown syntax

Reply in this format:
Front: [edited front]
Back: [edited back]

Terminal commands:
```bash
$ command <placeholder>
```

Code:
```language
code here
```

Use the following placeholders only: <file>, <path>, <link>, <command>.

No explanations.

Return results using this JSON schema:
{
    "title": "Note",
    "type": "object",
    "properties": {
        "front": {"type": "string"},
        "back": {"type": "string"},
    },
    "required": ["front", "back"]
}

Example 1:
Front: What command does extract files from a zip archive?
Back: ```bash
$ unzip <file>
```
{ "front": "Extract zip files", "back": "```bash<br>$ unzip <file><br>```" }

Example 2:
Front: What is the command to print manual or get help for a command?
Back: ```bash
$ man ...
```
{ "front": "Get command manual/help", "back": "```bash<br>$ man <command><br>```" }

Example 3: 
Front: What command does create a soft link?
Back: ```bash
$ ln -s <file_name> <link_name>
```
{ "front": "Create soft link", "back": "```bash<br>$ ln -s <file> <link><br>```" }

Example 4:
Front: In the `ln -s` command, what is the order of file name and link name?
Back: ```bash
$ ln -s <file_name> <link_name>
```
{ "front": "`ln -s` argument order", "back": "Back: <file> then <link>" }

Example 5:
Front: What is the range of the Leaky ReLU function?
Back: $ [ -0.01, + \infty ] $
{ "front": "Leaky ReLU range", "back": "$ [-0.01, +\infty] $" }
"""

user_msg = lambda note, _: f"""Front: {strip_tags(note.front)}\nBack: {strip_tags(note.back)}\nTags: {note.tags}"""

def format_notes(notes, chat) -> (Note, Note):
    return [apply_transform(system_msg, user_msg, note, chat, json_mode=True) for note in notes]

notes_bundle = format_notes(sample, chat)


#######################

Front: Motion to move the cursor [count] WORDS forward
Back: "`[count]W`

"
Tags: ['nvim']

Front: Motion: WORDS forward
Back: `[count]W`
Tags: ['nvim']

#######################

Front: "What type of tyres can a hooked rim fit?

"
Back: Clincher and (if specified) tubeless-ready tyres
Tags: ['cycling']

Front: Hooked rim tyre type
Back: Clincher and (if specified) tubeless-ready tyres
Tags: None

#######################

Front: What command line tool controls the Bluetooth  connections?
Back: `bluetoothctl`
Tags: ['linux']

Front: Bluetooth connection tool
Back: ```\
Tags: ['linux']

#######################

Front: What key returns the `^` in the shifted state?
Back: "`6`

"
Tags: ['keyboard']

Front: Shifted key for `^`
Back: `6`
Tags: ['keyboard']

#######################

Front: What does a high negative TSB number says?
Back: "The athlete is in ""the hole"""
Tags: ['triathlon']

Front: High negative TSB
Back: The athlete is in the hole
Tags: ['triathlon']


In [9]:
system_msg = """Code block must follows this format:

```<language>
<code>
```

Inline code blocks must follow this format:

`<code>`

Inline code blocks are often used for 1-line commands (e.g., vim, grep), to highlight libraries/tools names, or keymaps. If you see a `` block in the original note, assume it is used correctly.

If the note does not contain a code block, leave it unchanged. 
"""

user_msg = lambda note, edited_note: f"""Original note:\nFront: {strip_tags(note.front)}\nBack: {strip_tags(note.back)}\nTags: {note.tags}\n\nUpdated note:\nFront: {strip_tags(edited_note.front)}\nBack: {strip_tags(edited_note.back)}\nTags: {edited_note.tags}"""


def fix_code_blocks(notes, chat) -> Note:
    return [apply_transform(system_msg, user_msg, orig_note, edited_note, chat, json_mode=True) for (orig_note, edited_note) in notes]

notes_bundle = fix_code_blocks(notes_bundle, chat)


#######################

Front: Motion to move the cursor [count] WORDS forward
Back: "`[count]W`

"
Tags: ['nvim']

Front: Motion: WORDS forward
Back: [count]W
Tags: ['nvim']

#######################

Front: "What type of tyres can a hooked rim fit?

"
Back: Clincher and (if specified) tubeless-ready tyres
Tags: ['cycling']

Front: Hooked rim tyre type
Back: Clincher and (if specified) tubeless-ready tyres
Tags: []

#######################

Front: What command line tool controls the Bluetooth  connections?
Back: `bluetoothctl`
Tags: ['linux']

Front: Bluetooth connection tool
Back: ```\", 
Tags: ['linux']

#######################

Front: What key returns the `^` in the shifted state?
Back: "`6`

"
Tags: ['keyboard']

Front: Shifted key for `^`
Back: `6`
Tags: ['keyboard']

#######################

Front: What does a high negative TSB number says?
Back: "The athlete is in ""the hole"""
Tags: ['triathlon']

Front: High negative TSB
Back: The athlete is in the hole
Tags: ['triathlon']



In [10]:
# TODO: we should print the original note, the edited note, and the newly edited note.

In [11]:
def fix_math_blocks(original_note: Note, edited_note: Note) -> Note:
    pass

In [12]:
def fix_media_content(original_note: Note, edited_note: Note) -> Note:
    pass

In [13]:
def fix_html_tags(original_note: Note, edited_note: Note) -> Note:
    pass